In [9]:
import optuna
import gym
import numpy as np
import random
from q_learning_oop import *
from IPython.display import clear_output
from test_runner import *

frozen_lake_env = gym.make("FrozenLake-v0").env

In [10]:
def get_model_params(trial):
    """
    Learning hyperparamters we want to optimize
    """
    return {
          "alpha": trial.suggest_uniform("alpha", 0.1, 0.9),
          "gamma": trial.suggest_uniform("gamma", 0.9, 1),
          "min_epsilon": trial.suggest_uniform("min_epsilon", 0.01, 0.1),
          "epsilon_decay": trial.suggest_uniform("epsilon_decay", 0.8, 0.99999)
    }

In [ ]:
%%time
def objective(trial):
    """
    Define an objective function to be minimized.
    """
    model_params = get_model_params(trial)
    agent = QLearningAgent(frozen_lake_env, epsilon=1, **model_params)
    train_runner = TrainRunner(frozen_lake_env, agent)
    train_runner.train(episodes=5000, steps_per_episode=100)
    tester = TestRunner(frozen_lake_env, agent)
    test_results = tester.test(5000)
    return test_results.bad  # A objective value linked with the Trial object.

study = optuna.create_study()  # Create a new study.
optuna.logging.disable_default_handler()
study.optimize(objective, n_trials=50, n_jobs=-1)  # Invoke optimization of the objective function.
clear_output()
print(f"best:\n\t{study.best_params}")
print(f"trials = {len(study.trials)}")

In [ ]:
print("f")